In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.tools import Tool
from langchain_google_genai import ChatGoogleGenerativeAI

d:\LEARN\LangChain Basis\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
cd ..

d:\LEARN\LangChain Basis


d:\LEARN\LangChain Basis\.venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
# Define Tools
def get_current_time(*args, **kwargs):
    """Returns the current time in H:MM AM/PM format."""
    import datetime

    now = datetime.datetime.now()
    return now.strftime("%I:%M %p")


def search_wikipedia(query):
    """Searches Wikipedia and returns the summary of the first result."""
    from wikipedia import summary

    try:
        # Limit to two sentences for brevity
        return summary(query, sentences=2)
    except:
        return "I couldn't find any information on that."

In [5]:
# Define the tools that the agent can use
get_current_time_tool = Tool(
    name="Time",
    func=get_current_time,
    description="Useful for when you need to know the current time.",
)

search_wikipedia_tool = Tool(
    name="Wikipedia",
    func=search_wikipedia,
    description="Useful for when you need to know information about a topic.",
)

tools = [get_current_time_tool, search_wikipedia_tool]

In [6]:
# Load the correct JSON Chat Prompt from the hub
prompt = hub.pull("hwchase17/structured-chat-agent")

d:\LEARN\LangChain Basis\.venv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [17]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

In [27]:
# Create a structured Chat Agent with Conversation Buffer Memory
# ConversationBufferMemory stores the conversation history, allowing the agent to maintain context across interactions
memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True
)

In [28]:
# Initial system message to set the context for the chat
# SystemMessage is used to define a message from the system to the agent, setting initial instructions or context
# initial_message = "You are an AI assistant that can provide helpful answers using available tools.\nIf you are unable to answer, you can use the following tools: Time and Wikipedia."
# memory.chat_memory.add_message(SystemMessage(content=initial_message))

In [29]:
# create_structured_chat_agent initializes a chat agent designed to interact using a structured prompt and tools
# It combines the language model (llm), tools, and prompt to create an interactive agent
agent = create_structured_chat_agent(
    llm=llm, 
    tools=tools, 
    prompt=prompt
)

In [30]:
# AgentExecutor is responsible for managing the interaction between the user input, the agent, and the tools
# It also handles memory to ensure context is maintained throughout the conversation
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,  # Use the conversation memory to maintain context
    handle_parsing_errors=True,  # Handle any parsing errors gracefully
)

In [31]:
# Chat Loop to interact with the user
while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        break

    # Add the user's message to the conversation memory
    memory.chat_memory.add_message(HumanMessage(content=user_input))

    # Invoke the agent with the user input and the current chat history
    response = agent_executor.invoke({"input": user_input})
    print("Bot:", response["output"])

    # Add the agent's response to the conversation memory
    memory.chat_memory.add_message(AIMessage(content=response["output"]))



> Entering new AgentExecutor chain...
[SystemMessage(content='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nTime(*args, **kwargs) - Useful for when you need to know the current time., args: {\'tool_input\': {\'type\': \'string\'}}\nWikipedia(query) - Useful for when you need to know information about a topic., args: {\'tool_input\': {\'type\': \'string\'}}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or Time, Wikipedia\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{\n  "action": "Fi

In [27]:
memory.chat_memory.messages

[SystemMessage(content='You are an AI assistant that can provide helpful answers using available tools.\nIf you are unable to answer, you can use the following tools: Time and Wikipedia.'),
 HumanMessage(content='hi'),
 SystemMessage(content='Goodbye!')]